In [1]:
import numpy as np
from numba import njit

In [2]:
def read_input(filename):
    
    with open(filename) as f:
        input_text = f.readlines()

    input_list = []

    for line in input_text:
        line_parts = line.split()
        sensor_j = int(line_parts[2].split("=")[1][:-1])
        sensor_i = int(line_parts[3].split("=")[1][:-1])
        sensor = (sensor_i, sensor_j)
        beacon_j = int(line_parts[8].split("=")[1][:-1])
        beacon_i = int(line_parts[9].split("=")[1])
        beacon = (beacon_i, beacon_j)

        input_list.append((sensor, beacon))
        
    return input_list


@njit
def manhattan_distance(pos_1, pos_2):
    
    return abs(pos_1[0]-pos_2[0]) + abs(pos_1[1] - pos_2[1])


@njit
def manhattan_distance_dict(d):
    
    pos_1 = d[0]
    pos_2 = d[1]
    
    return manhattan_distance(pos_1, pos_2)


def find_dim(input_list):
    
    min_x = np.inf
    max_x = -np.inf
    max_dist = 0
    
    for obs in input_list:
        sensor_j = obs[0][1]
        dist = manhattan_distance_dict(obs)
        if sensor_j < min_x:
            min_x = sensor_j
        if sensor_j > max_x:
            max_x = sensor_j
        if dist > max_dist:
            max_dist = dist
            
    return min_x, max_x, max_dist


def count_no_beacon_in_line(i_rel, input_list):

    min_x, max_x, max_dist = find_dim(input_list)

    line_rel = np.zeros(max_x - min_x + 2*max_dist + 1, dtype="bool")

    for obs in input_list:
        dist = manhattan_distance_dict(obs)
        sensor_i = obs[0][0]
        sensor_j = obs[0][1]
        obs_rel_below = sensor_i >= i_rel and sensor_i - dist <= i_rel
        obs_rel_above = sensor_i <= i_rel and sensor_i + dist >= i_rel
        obs_rel = obs_rel_below or obs_rel_above

        if obs_rel:
            dist_rem = dist - abs(sensor_i - i_rel)
            line_rel[sensor_j - dist_rem - (min_x - max_dist): sensor_j + dist_rem + 1  - (min_x - max_dist)] = True
            
    for obs in input_list:
        beacon_i, beacon_j = obs[1]
        if beacon_i == i_rel:
            line_rel[beacon_j - (min_x - max_dist)] = False
            
    return line_rel.sum()

## Part I

In [3]:
input_list_test = read_input("15_test.txt")
count_no_beacon_in_line(10, input_list_test)

26

In [4]:
input_list = read_input("15.txt")
count_no_beacon_in_line(2000000, input_list)

4886370

## Part II

In [5]:
@njit
def count_no_beacon_in_line_2(i_rel, input_list, max_x):

    line_rel = np.zeros(max_x + 1, dtype=np.bool_)

    for obs in input_list:
        dist = manhattan_distance_dict(obs)
        sensor_i, sensor_j = obs[0]
        dist_rem = dist - abs(sensor_i - i_rel)

        if dist_rem >= 0:
            x_l = np.fmax(0, sensor_j - dist_rem)
            x_h = np.fmin(max_x, sensor_j + dist_rem)
            line_rel[x_l: x_h + 1] = True
            
    if line_rel.sum() < max_x + 1:
        return np.argmin(line_rel)
    else:
        return -1

In [6]:
import time

In [7]:
@njit
def no_beacon(input_list, start_x, max_x):
    for i in range(start_x, max_x):
        #if i % 1000 == 0: print(i)
        res = count_no_beacon_in_line_2(i, input_list, max_x)
        if res != -1:
            return res*4000000 + i

In [8]:
no_beacon(input_list_test, 0, 20)

<ipython-input-7-cc7dddb9f919>:5: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'input_list' of function 'count_no_beacon_in_line_2'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-5-6b9ffd48b18b>", line 2:
@njit
def count_no_beacon_in_line_2(i_rel, input_list, max_x):
^

  res = count_no_beacon_in_line_2(i, input_list, max_x)
C:\Users\djschmi\Anaconda3\lib\site-packages\numba\core\ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'input_list' of function 'no_beacon'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-7-cc7dddb9f919>", line 2:
@njit
def no

56000011

In [11]:
t0 = time.time()
start_x = 2500000
max_x   = 3000000
#4000000
res = no_beacon(input_list, start_x, max_x)
print(time.time() - t0)
print(res)

584.8641958236694
11374534948438
